In [2]:
import torch
from torch import nn
import torch.nn.functional as F

class My_Encoder(nn.Module):
    def __init__(self, dim_encoder_output, activation='tanh'):
        super().__init__()
        if activation == "tanh":
            self.activation = torch.tanh
        self.l0 = nn.Linear(4, 4)
        self.l1 = nn.Linear(4, dim_encoder_output)

    def forward(self, x):
        h = self.activation(self.l0(x))
        z = self.l1(h)
        return z


class My_Decoder(nn.Module):
    def __init__(self, dim_decoder_input, activation='tanh'):
        super().__init__()
        if activation == "tanh":
            self.activation = torch.tanh
        self.l0 = nn.Linear(dim_decoder_input, 4)
        self.l1 = nn.Linear(4, 4)

    def forward(self, z):
        x_hat = self.activation(self.l0(z))
        x_hat = self.l1(x_hat)
        return x_hat

In [4]:
from collections import OrderedDict

od = OrderedDict([
    ('encoder',My_Encoder(2)),
    ('decoder',My_Decoder(2))
])

model = nn.Sequential(od)

In [5]:
print(model)

Sequential(
  (encoder): My_Encoder(
    (l0): Linear(in_features=4, out_features=4, bias=True)
    (l1): Linear(in_features=4, out_features=2, bias=True)
  )
  (decoder): My_Decoder(
    (l0): Linear(in_features=2, out_features=4, bias=True)
    (l1): Linear(in_features=4, out_features=4, bias=True)
  )
)


In [7]:
from sklearn.datasets import load_iris

iris = load_iris()
print(type(iris))

<class 'sklearn.utils._bunch.Bunch'>


In [9]:
print(type(iris.data))
print(iris.data.shape)

<class 'numpy.ndarray'>
(150, 4)


In [11]:
for p in model.get_submodule('encoder').parameters():
    print(p)

Parameter containing:
tensor([[-0.3029, -0.1730,  0.1353, -0.0692],
        [ 0.3013, -0.4437, -0.0029, -0.4157],
        [ 0.3386,  0.0310, -0.2414, -0.0330],
        [ 0.3433, -0.2415, -0.4646,  0.1265]], requires_grad=True)
Parameter containing:
tensor([ 0.2804, -0.2436,  0.3656, -0.0853], requires_grad=True)
Parameter containing:
tensor([[ 0.3535,  0.1539,  0.2257, -0.1388],
        [-0.0791, -0.2168,  0.4046,  0.1868]], requires_grad=True)
Parameter containing:
tensor([-0.2449, -0.2655], requires_grad=True)


In [15]:
from torch.utils.data import TensorDataset, DataLoader

x = torch.Tensor(iris.data)

my_dataset = TensorDataset(x, x) # create your datset
my_dataloader = DataLoader(my_dataset, batch_size=30, shuffle=True) # create your dataloader

In [16]:
optimizer = torch.optim.Adam(model.parameters())

for idx in range(5):
    for x, y in my_dataloader:
        pred = model(x)

        loss = (pred - y).square().sum() / len(x)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [17]:
for p in model.get_submodule('encoder').parameters():
    print(p)

Parameter containing:
tensor([[-0.2881, -0.1586,  0.1506, -0.0533],
        [ 0.3260, -0.4189,  0.0218, -0.3910],
        [ 0.3125,  0.0048, -0.2675, -0.0591],
        [ 0.3189, -0.2661, -0.4888,  0.1024]], requires_grad=True)
Parameter containing:
tensor([ 0.2949, -0.2189,  0.3394, -0.1098], requires_grad=True)
Parameter containing:
tensor([[ 0.3803,  0.1775,  0.1989, -0.1122],
        [-0.0540, -0.1961,  0.3797,  0.2124]], requires_grad=True)
Parameter containing:
tensor([-0.2717, -0.2907], requires_grad=True)


In [19]:
for p in model.get_submodule('decoder').parameters():
    print(p)

Parameter containing:
tensor([[-0.0406, -0.5920],
        [ 0.2950,  0.3930],
        [-0.0942, -0.0262],
        [ 0.4880,  0.4786]], requires_grad=True)
Parameter containing:
tensor([ 0.3218, -0.2653,  0.3969, -0.0916], requires_grad=True)
Parameter containing:
tensor([[ 0.2523, -0.4001, -0.0030,  0.1688],
        [-0.3429, -0.3876, -0.0260,  0.2116],
        [ 0.4485,  0.1572,  0.1299, -0.3265],
        [-0.1425,  0.1972,  0.3776,  0.4246]], requires_grad=True)
Parameter containing:
tensor([-0.3792, -0.1653, -0.3221,  0.0607], requires_grad=True)


In [18]:
for p in model.get_submodule('encoder').parameters():
    p.requires_grad_(False)

In [20]:
for idx in range(5):
    for x, y in my_dataloader:
        pred = model(x)

        loss = (pred - y).square().sum() / len(x)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [21]:
for p in model.get_submodule('encoder').parameters():
    print(p)
for p in model.get_submodule('decoder').parameters():
    print(p)

Parameter containing:
tensor([[-0.2927, -0.1633,  0.1465, -0.0569],
        [ 0.3349, -0.4100,  0.0307, -0.3820],
        [ 0.3020, -0.0057, -0.2780, -0.0696],
        [ 0.3102, -0.2748, -0.4973,  0.0940]])
Parameter containing:
tensor([ 0.2902, -0.2100,  0.3290, -0.1185])
Parameter containing:
tensor([[ 0.3913,  0.1818,  0.1880, -0.1010],
        [-0.0447, -0.1924,  0.3705,  0.2224]])
Parameter containing:
tensor([-0.2827, -0.3001])
Parameter containing:
tensor([[-0.0670, -0.6208],
        [ 0.3209,  0.4093],
        [-0.1237, -0.0587],
        [ 0.4664,  0.4708]], requires_grad=True)
Parameter containing:
tensor([ 0.3473, -0.2905,  0.4254, -0.0709], requires_grad=True)
Parameter containing:
tensor([[ 2.8106e-01, -4.2777e-01,  2.2757e-02,  1.4150e-01],
        [-3.1408e-01, -4.1535e-01, -1.7625e-04,  1.8435e-01],
        [ 4.7696e-01,  1.2965e-01,  1.5551e-01, -3.5375e-01],
        [-1.1386e-01,  1.6946e-01,  4.0353e-01,  3.9710e-01]],
       requires_grad=True)
Parameter containing:


In [22]:
optimizer = torch.optim.Adam(model.parameters())

for idx in range(5):
    for x, y in my_dataloader:
        pred = model(x)

        loss = (pred - y).square().sum() / len(x)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

for p in model.get_submodule('encoder').parameters():
    print(p)
for p in model.get_submodule('decoder').parameters():
    print(p)

Parameter containing:
tensor([[-0.2927, -0.1633,  0.1465, -0.0569],
        [ 0.3349, -0.4100,  0.0307, -0.3820],
        [ 0.3020, -0.0057, -0.2780, -0.0696],
        [ 0.3102, -0.2748, -0.4973,  0.0940]])
Parameter containing:
tensor([ 0.2902, -0.2100,  0.3290, -0.1185])
Parameter containing:
tensor([[ 0.3913,  0.1818,  0.1880, -0.1010],
        [-0.0447, -0.1924,  0.3705,  0.2224]])
Parameter containing:
tensor([-0.2827, -0.3001])
Parameter containing:
tensor([[-0.0921, -0.6457],
        [ 0.3460,  0.4341],
        [-0.1489, -0.0838],
        [ 0.4431,  0.4512]], requires_grad=True)
Parameter containing:
tensor([ 0.3724, -0.3156,  0.4507, -0.0476], requires_grad=True)
Parameter containing:
tensor([[ 0.3061, -0.4528,  0.0478,  0.1168],
        [-0.2890, -0.4404,  0.0249,  0.1596],
        [ 0.5019,  0.1047,  0.1805, -0.3784],
        [-0.0891,  0.1447,  0.4284,  0.3726]], requires_grad=True)
Parameter containing:
tensor([-0.3294, -0.1155, -0.2726,  0.1104], requires_grad=True)
